In [88]:
___Author___='LumberJack Jyss'

In [89]:
print('Global Optimized LumberJack Equities Method\n°OoO_FOREX_Oo0°\nLumberJack Jyss 5779(c)')

Global Optimized LumberJack Equities Method
°OoO_FOREX_Oo0°
LumberJack Jyss 5779(c)


In [90]:
print('Importing librairies...')
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score,roc_curve,confusion_matrix,classification_report
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import to_categorical
%matplotlib inline
print('Librairies imported')

Importing librairies...
Librairies imported


In [91]:
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore")
tf.logging.set_verbosity(tf.logging.ERROR)

In [92]:
df_1D = pd.read_csv('20190526 data prepared.csv')
df_1D = df_1D.iloc[:,1:]
df_1D = df_1D.dropna()
df_1D.reset_index(inplace=True,drop=True)

FileNotFoundError: [Errno 2] File b'20190526 data prepared.csv' does not exist: b'20190526 data prepared.csv'

In [ ]:
df_1D.isnull().sum().sum()

In [ ]:
df_1D.info()

In [ ]:
plt.figure(figsize=(26,20))
sns.heatmap(df_1D.iloc[:,:-1].corr(),annot=True,cmap='rainbow')
plt.show()

In [ ]:
df_1D.shape

In [ ]:
df_1D = df_1D.drop(['target_up','target_down'],axis=1)

In [ ]:
delta = round(df_1D.shape[0])
bloc1 = round(delta*0.8)
bloc2 = delta - bloc1
print("# of periods : ",delta)
print('On 80% - 20% slash : ')
print('Bloc 1 : ',bloc1,' periods (',df_1D.iloc[bloc1,0],')\nBloc 2 :',bloc2,' periods (',df_1D.iloc[bloc2,0],')')
print('First period :',df_1D.iloc[0,0])
print('Last period :',df_1D.iloc[df_1D.shape[0]-1,0])


In [ ]:
X = df_1D.iloc[:,1:-1]
y = df_1D.iloc[:,-1].shift(-1).values

In [ ]:
X.info()

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
y = np.array(y).reshape(-1,1)

In [ ]:
Xtrain = X[:bloc1,:]
Xtest = X[bloc1:,:]
ytrain = y[:bloc1,:]
ytest = y[bloc1:,:]

In [ ]:
Xtrain.shape,Xtest.shape,ytrain.shape,ytest.shape

In [ ]:
# fix random seed for reproducibility
seed = 770
np.random.seed(seed)

In [ ]:
ytrain_= ytrain.reshape(ytrain.shape[0],)
Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1])

model = Sequential()
# Add an input layer 
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dropout(0.2))

# Add one hidden layer 
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))


# Add an output layer 
model.add(Dense(1, activation='relu'))

In [ ]:
print('Processing prediction')
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mae','rmse'])
                   
history = model.fit(Xtrain, ytrain,epochs=50, batch_size=42, verbose=0)

print('Computing done')

In [ ]:
train_acc = model.evaluate(Xtrain, ytrain,verbose=1)
train_acc

In [ ]:
yhat = model.predict(Xtest)

In [ ]:
score = model.evaluate(Xtest, ytest,verbose=1)
score

In [ ]:
ytrue = df_1D.iloc[-1288:,-1]
ytrue.reset_index(inplace=True,drop=True)
ytrue.shape,len(yhat),df_1D.iloc[-1288:,-1].shape

In [ ]:
resultats.head()

In [ ]:
resultats = pd.DataFrame()
resultats['Date'] = df_1D.iloc[-1288:,0]
resultats.reset_index(inplace=True,drop=True)
resultats['Actual'] = ytrue
resultats['Predicted'] = yhat

In [ ]:
plt.figure(figsize=(26,8),dpi=300)
plt.plot(ytrue,color='blue',lw=0.8)
plt.plot(yhat,color='orange',label='predicted',lw=0.8)
#plt.xlim(5100,6500)
#plt.ylim(2700,3000)
plt.legend()

In [ ]:
# save the model to disk
filename = 'finalized_model_prediction.sav'
pickle.dump(model, open(filename, 'wb'))


# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

##### 